# AFL player match stats - wrangling
I am interested in analysing publically available AFL data to look for insights into brownlow votes. First, I need to access player match data and wrangle it together. I began with data available from [AFL Tables](https://afltables.com/afl/afl_index.html), then supplemented it with some additional data from [Footywire](https://www.footywire.com/).

## Import libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

from bs4 import BeautifulSoup
import requests
import urllib.parse
import numpy as np

## <u>Table of Contents</u> <a id='L'></a>

<font size = 4><b>[1. AFL Tables](#1)</b></font> <br>
&nbsp;&nbsp; <b>[1.1 Read and clean tables for one game](#1.1)</b> <br>
&nbsp;&nbsp;&nbsp;&nbsp; [1.2.1 Retrieve web links for 2022 games](#1.2.1) <br>
&nbsp;&nbsp;&nbsp;&nbsp; [1.2.2 Retrieve web links for 2015-2022 games](#1.2.2) <br>
&nbsp;&nbsp; <b>[1.2 Retrieve web links](#1.2)</b> <br>
&nbsp;&nbsp; <b>[1.3 Wrangle many tables](#1.3)</b> <br>
<font size = 4><b>[2. Footywire](#2)</b></font> <br>
&nbsp;&nbsp; <b>[2.1 Read and clean tables for one game](#2.1)</b> <br>
&nbsp;&nbsp; <b>[2.2 Retrieve web links](#2.2)</b> <br>
&nbsp;&nbsp;&nbsp;&nbsp; [2.2.1 Retrieve web links for 2022 games](#2.2.1) <br>
&nbsp;&nbsp;&nbsp;&nbsp; [2.2.2 Retrieve web links for 2015-2022 games](#2.2.2) <br>
&nbsp;&nbsp; <b>[2.3 Wrangle many tables](#2.3)</b> <br>
<font size = 4><b>[3. Merge AFL tables and Footywire dataframes](#3)</b></font> <br>



## [1. AFL Tables](#L) <a id='1'></a>

### [1.1 Read and clean tables for one game](#L) <a id='1.1'></a>

In [2]:
pd.set_option('display.max_columns', None)

# Read sample game - first game of 2022 between Melbourne and Western Bulldogs
df = pd.read_html("https://afltables.com/afl/stats/games/2022/071120220316.html")

When we read a page using pandas, it will return a list of dataframes, but we only want the two tables of players stats, one for each team. These tables will typically be at index 2 and 4 of the list.

In [3]:
# Preview the table for the home team in the match
df[2].head()

Melbourne Match Statistics [Season][Game by Game]                          \
                                                  #           Player  KI MK   
0                                              12 ↑    Bedford, Toby   5  3   
1                                                17      Bowey, Jake   8  2   
2                                                10  Brayshaw, Angus  12  6   
3                                                50       Brown, Ben   9  8   
4                                                31  Fritsch, Bayley   8  4   

                                                                           \
   HB  DI   GL   BH  HO   TK   RB   IF   CL   CG   FF   FA  BR   CP    UP   
0   4   9  NaN  1.0 NaN  2.0  1.0  1.0  NaN  1.0  2.0  NaN NaN  4.0   5.0   
1   1   9    1  NaN NaN  NaN  2.0  1.0  1.0  1.0  NaN  NaN NaN  3.0   6.0   
2  11  23  NaN  NaN NaN  3.0  2.0  2.0  NaN  4.0  1.0  1.0 NaN  4.0  19.0   
3   4  13    3  3.0 NaN  NaN  NaN  1.0  NaN  1.0  NaN  1.0 NaN  6.0   7.0   
4   1   9    2  2.0 NaN  1.0  NaN  2.0  NaN  2.0  NaN  1.0 NaN  3.0   5.0   

                                  
    CM   MI   1%   BO   GA    %P  
0  NaN  1.0  2.0  1.0  NaN  55.0  
1  NaN  NaN  2.0  NaN  NaN  73.0  
2  NaN  NaN  1.0  NaN  NaN  83.0  
3  2.0  6.0  2.0  NaN  NaN  86.0  
4  1.0  3.0  NaN  NaN  1.0  81.0

There are a few things that we will want to clean up such as the NaN values. It is also worth noting that the the player stats tables have a MultiIndex column (e.g., "Melbourne Match Statistics [Season][Game by Game]", and then each column name).

In [4]:
type(df[2].columns)

pandas.core.indexes.multi.MultiIndex

Let's clean up the table by dropping the first row of headings as it will not be needed

In [5]:
# Drop top table heading
df[2] = df[2].droplevel(0, axis=1) # Melbourne
df[4] = df[4].droplevel(0, axis=1) # Western Bulldogs

In [6]:
df[2].head()

,#,Player,KI,MK,HB,DI,GL,BH,HO,TK,RB,IF,CL,CG,FF,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P
0,12 ↑,"Bedford, Toby",5,3,4,9,NaN,1.0,NaN,2.0,1.0,1.0,NaN,1.0,2.0,NaN,NaN,4.0,5.0,NaN,1.0,2.0,1.0,NaN,55.0
1,17,"Bowey, Jake",8,2,1,9,1,NaN,NaN,NaN,2.0,1.0,1.0,1.0,NaN,NaN,NaN,3.0,6.0,NaN,NaN,2.0,NaN,NaN,73.0
2,10,"Brayshaw, Angus",12,6,11,23,NaN,NaN,NaN,3.0,2.0,2.0,NaN,4.0,1.0,1.0,NaN,4.0,19.0,NaN,NaN,1.0,NaN,NaN,83.0
3,50,"Brown, Ben",9,8,4,13,3,3.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,6.0,7.0,2.0,6.0,2.0,NaN,NaN,86.0
4,31,"Fritsch, Bayley",8,4,1,9,2,2.0,NaN,1.0,NaN,2.0,NaN,2.0,NaN,1.0,NaN,3.0,5.0,1.0,3.0,NaN,NaN,1.0,81.0


We can include columns for club name, opposition club, round and year to help with identifying matches.

In [7]:
# Record club name in each row
df[2]['Club'] = df[0][1][1]
df[4]['Club'] = df[0][1][2]

# Record opponent club name in each row
df[2]['Opponent'] = df[0][1][2]
df[4]['Opponent'] = df[0][1][1]

# Record year and round in each row
df[2]['Round'] = df[4]['Round'] = df[0][1][0].split('Round: ')[1].split(" Venue")[0]
df[2]['Year'] = df[4]['Year'] = df[0][1][0].split('-')[2].split(' ')[0]

In [8]:
# Check the dataframe for team 2: Western Bulldogs
df[4]

,#,Player,KI,MK,HB,DI,GL,BH,HO,TK,RB,IF,CL,CG,FF,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P,Club,Opponent,Round,Year
0,4,"Bontempelli, Marcus",13,7,4,17,1,NaN,NaN,7.0,3.0,4.0,1.0,3.0,1.0,1.0,NaN,6.0,12.0,1.0,NaN,3.0,NaN,2.0,72.0,Western Bulldogs,Melbourne,1,2022
1,12,"Cordy, Zaine",1,NaN,2,3,NaN,NaN,1.0,3.0,NaN,NaN,1.0,2.0,1.0,2.0,NaN,1.0,1.0,NaN,NaN,4.0,NaN,1.0,84.0,Western Bulldogs,Melbourne,1,2022
2,9,"Crozier, Hayden",14,12,5,19,1,NaN,NaN,3.0,3.0,2.0,NaN,3.0,NaN,2.0,NaN,3.0,18.0,1.0,NaN,2.0,1.0,NaN,75.0,Western Bulldogs,Melbourne,1,2022
3,31,"Dale, Bailey",17,6,8,25,1,NaN,NaN,NaN,6.0,3.0,1.0,2.0,1.0,NaN,NaN,4.0,17.0,NaN,NaN,2.0,NaN,NaN,83.0,Western Bulldogs,Melbourne,1,2022
4,35,"Daniel, Caleb",14,6,12,26,NaN,NaN,NaN,2.0,7.0,2.0,NaN,2.0,NaN,1.0,NaN,5.0,17.0,NaN,NaN,1.0,2.0,NaN,80.0,Western Bulldogs,Melbourne,1,2022
5,5,"Dunkley, Josh",14,4,15,29,NaN,1.0,NaN,5.0,2.0,4.0,3.0,3.0,1.0,1.0,NaN,12.0,17.0,NaN,NaN,3.0,1.0,1.0,84.0,Western Bulldogs,Melbourne,1,2022
6,44,"English, Tim",16,6,4,20,NaN,1.0,18.0,2.0,4.0,1.0,8.0,2.0,8.0,1.0,NaN,13.0,7.0,2.0,1.0,2.0,NaN,NaN,86.0,Western Bulldogs,Melbourne,1,2022
7,43,"Gardner, Ryan",6,3,1,7,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,6.0,NaN,NaN,5.0,NaN,NaN,100.0,Western Bulldogs,Melbourne,1,2022
8,29,"Hannan, Mitch",5,NaN,3,8,1,NaN,NaN,NaN,NaN,1.0,NaN,3.0,1.0,NaN,NaN,6.0,2.0,NaN,NaN,NaN,NaN,NaN,72.0,Western Bulldogs,Melbourne,1,2022
9,7,"Hunter, Lachie",7,4,6,13,NaN,NaN,NaN,3.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,3.0,10.0,NaN,NaN,2.0,NaN,NaN,88.0,Western Bulldogs,Melbourne,1,2022


In [9]:
# Add result: whether the team won, lost or drew
team1_score = df[0][5][1].split('.')[2]
team2_score = df[0][5][2].split('.')[2]

if team1_score > team2_score:
    df[2]['Result'] = 'W'
    df[4]['Result'] = 'L'
elif team2_score > team1_score:
    df[2]['Result'] = 'L'
    df[4]['Result'] = 'W'
elif team1_score == team2_score:
    df[2]['Result'] = 'D'
    df[4]['Result'] = 'D'

In [10]:
# Add game margin
df[2]['Margin'] = int(team1_score) - int(team2_score)
df[4]['Margin'] = int(team2_score) - int(team1_score)

There are also soms rows at the bottom of the table that we don't need: rushed behinds and stat totals for each team.

In [11]:
# Check the bottom of the table
df[2].tail()

,#,Player,KI,MK,HB,DI,GL,BH,HO,TK,RB,IF,CL,CG,FF,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P,Club,Opponent,Round,Year,Result,Margin
21,20,"Tomlinson, Adam",6,4,2,8,NaN,NaN,NaN,1.0,1.0,NaN,NaN,2.0,NaN,1.0,NaN,3.0,6.0,NaN,NaN,4.0,NaN,NaN,100.0,Melbourne,Western Bulldogs,1,2022,W,26
22,7,"Viney, Jack",11,5,12,23,1,NaN,NaN,4.0,2.0,3.0,4.0,4.0,2.0,NaN,NaN,12.0,11.0,NaN,1.0,1.0,NaN,NaN,79.0,Melbourne,Western Bulldogs,1,2022,W,26
23,Rushed,Rushed,Rushed,Rushed,Rushed,Rushed,Rushed,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Melbourne,Western Bulldogs,1,2022,W,26
24,Totals,Totals,213,96,137,350,14,13.0,44.0,46.0,40.0,56.0,30.0,72.0,17.0,30.0,4.0,146.0,204.0,16.0,17.0,54.0,8.0,8.0,NaN,Melbourne,Western Bulldogs,1,2022,W,26
25,Opposition,Opposition,224,92,152,376,11,5.0,20.0,53.0,43.0,51.0,40.0,59.0,30.0,17.0,2.0,126.0,243.0,6.0,6.0,43.0,4.0,8.0,NaN,Melbourne,Western Bulldogs,1,2022,W,26


In [12]:
# Replace NaN with 0s
df[2] = df[2].fillna(0)
df[4] = df[4].fillna(0)

In [13]:
# Drop 'Rushed behind' row and totals rows from the bottom
df[2] = df[2][(df[2]['#'] != 'Rushed') & (df[2]['#'] != 'Totals') & (df[2]['#'] != 'Opposition')]
df[4] = df[4][(df[4]['#'] != 'Rushed') & (df[4]['#'] != 'Totals') & (df[4]['#'] != 'Opposition')]

In [14]:
df[2]

,#,Player,KI,MK,HB,DI,GL,BH,HO,TK,RB,IF,CL,CG,FF,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P,Club,Opponent,Round,Year,Result,Margin
0,12 ↑,"Bedford, Toby",5,3,4,9,0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,4.0,5.0,0.0,1.0,2.0,1.0,0.0,55.0,Melbourne,Western Bulldogs,1,2022,W,26
1,17,"Bowey, Jake",8,2,1,9,1,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,6.0,0.0,0.0,2.0,0.0,0.0,73.0,Melbourne,Western Bulldogs,1,2022,W,26
2,10,"Brayshaw, Angus",12,6,11,23,0,0.0,0.0,3.0,2.0,2.0,0.0,4.0,1.0,1.0,0.0,4.0,19.0,0.0,0.0,1.0,0.0,0.0,83.0,Melbourne,Western Bulldogs,1,2022,W,26
3,50,"Brown, Ben",9,8,4,13,3,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,6.0,7.0,2.0,6.0,2.0,0.0,0.0,86.0,Melbourne,Western Bulldogs,1,2022,W,26
4,31,"Fritsch, Bayley",8,4,1,9,2,2.0,0.0,1.0,0.0,2.0,0.0,2.0,0.0,1.0,0.0,3.0,5.0,1.0,3.0,0.0,0.0,1.0,81.0,Melbourne,Western Bulldogs,1,2022,W,26
5,11,"Gawn, Max",11,4,3,14,1,0.0,34.0,1.0,2.0,5.0,4.0,9.0,2.0,6.0,0.0,12.0,3.0,2.0,0.0,3.0,0.0,0.0,97.0,Melbourne,Western Bulldogs,1,2022,W,26
6,4,"Harmes, James",6,1,11,17,1,0.0,0.0,5.0,1.0,3.0,2.0,4.0,2.0,1.0,0.0,7.0,10.0,1.0,1.0,3.0,0.0,0.0,77.0,Melbourne,Western Bulldogs,1,2022,W,26
7,29,"Hunt, Jayden",7,4,4,11,0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,7.0,1.0,0.0,2.0,0.0,0.0,86.0,Melbourne,Western Bulldogs,1,2022,W,26
8,6,"Jackson, Luke",3,5,9,12,1,1.0,10.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,8.0,5.0,2.0,1.0,1.0,0.0,0.0,86.0,Melbourne,Western Bulldogs,1,2022,W,26
9,23,"Jordon, James",10,4,10,20,0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,3.0,0.0,0.0,8.0,12.0,0.0,0.0,0.0,0.0,0.0,70.0,Melbourne,Western Bulldogs,1,2022,W,26


In [15]:
df[4]

,#,Player,KI,MK,HB,DI,GL,BH,HO,TK,RB,IF,CL,CG,FF,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P,Club,Opponent,Round,Year,Result,Margin
0,4,"Bontempelli, Marcus",13,7,4,17,1,0.0,0.0,7.0,3.0,4.0,1.0,3.0,1.0,1.0,0.0,6.0,12.0,1.0,0.0,3.0,0.0,2.0,72.0,Western Bulldogs,Melbourne,1,2022,L,-26
1,12,"Cordy, Zaine",1,0,2,3,0,0.0,1.0,3.0,0.0,0.0,1.0,2.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,1.0,84.0,Western Bulldogs,Melbourne,1,2022,L,-26
2,9,"Crozier, Hayden",14,12,5,19,1,0.0,0.0,3.0,3.0,2.0,0.0,3.0,0.0,2.0,0.0,3.0,18.0,1.0,0.0,2.0,1.0,0.0,75.0,Western Bulldogs,Melbourne,1,2022,L,-26
3,31,"Dale, Bailey",17,6,8,25,1,0.0,0.0,0.0,6.0,3.0,1.0,2.0,1.0,0.0,0.0,4.0,17.0,0.0,0.0,2.0,0.0,0.0,83.0,Western Bulldogs,Melbourne,1,2022,L,-26
4,35,"Daniel, Caleb",14,6,12,26,0,0.0,0.0,2.0,7.0,2.0,0.0,2.0,0.0,1.0,0.0,5.0,17.0,0.0,0.0,1.0,2.0,0.0,80.0,Western Bulldogs,Melbourne,1,2022,L,-26
5,5,"Dunkley, Josh",14,4,15,29,0,1.0,0.0,5.0,2.0,4.0,3.0,3.0,1.0,1.0,0.0,12.0,17.0,0.0,0.0,3.0,1.0,1.0,84.0,Western Bulldogs,Melbourne,1,2022,L,-26
6,44,"English, Tim",16,6,4,20,0,1.0,18.0,2.0,4.0,1.0,8.0,2.0,8.0,1.0,0.0,13.0,7.0,2.0,1.0,2.0,0.0,0.0,86.0,Western Bulldogs,Melbourne,1,2022,L,-26
7,43,"Gardner, Ryan",6,3,1,7,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,6.0,0.0,0.0,5.0,0.0,0.0,100.0,Western Bulldogs,Melbourne,1,2022,L,-26
8,29,"Hannan, Mitch",5,0,3,8,1,0.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,0.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,72.0,Western Bulldogs,Melbourne,1,2022,L,-26
9,7,"Hunter, Lachie",7,4,6,13,0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,10.0,0.0,0.0,2.0,0.0,0.0,88.0,Western Bulldogs,Melbourne,1,2022,L,-26


In [16]:
# Combine both teams
df_concat = pd.concat([df[2], df[4]])

We will also alter the format of player names to match another dataset. We will need to merge the two, so the format of the player names needs to be consistent. We will only keep the first initial, and the last name (partially shortened for hyphenated last names.

In [17]:
# Change format to initial of first name, followed by a space, then last name
df_concat['Player'] = df_concat['Player'].apply(lambda x: f"{x.split(', ')[1][0]} {x.split(', ')[0]}")
# Change format of hyphenated last names to initialise first component
df_concat['Player'] = df_concat['Player'].apply(lambda x: f"{x.split('-')[0][:3]}-{x.split('-')[1]}" if '-' in x else x)

In [18]:
df_concat

,#,Player,KI,MK,HB,DI,GL,BH,HO,TK,RB,IF,CL,CG,FF,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P,Club,Opponent,Round,Year,Result,Margin
0,12 ↑,T Bedford,5,3,4,9,0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,4.0,5.0,0.0,1.0,2.0,1.0,0.0,55.0,Melbourne,Western Bulldogs,1,2022,W,26
1,17,J Bowey,8,2,1,9,1,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,6.0,0.0,0.0,2.0,0.0,0.0,73.0,Melbourne,Western Bulldogs,1,2022,W,26
2,10,A Brayshaw,12,6,11,23,0,0.0,0.0,3.0,2.0,2.0,0.0,4.0,1.0,1.0,0.0,4.0,19.0,0.0,0.0,1.0,0.0,0.0,83.0,Melbourne,Western Bulldogs,1,2022,W,26
3,50,B Brown,9,8,4,13,3,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,6.0,7.0,2.0,6.0,2.0,0.0,0.0,86.0,Melbourne,Western Bulldogs,1,2022,W,26
4,31,B Fritsch,8,4,1,9,2,2.0,0.0,1.0,0.0,2.0,0.0,2.0,0.0,1.0,0.0,3.0,5.0,1.0,3.0,0.0,0.0,1.0,81.0,Melbourne,Western Bulldogs,1,2022,W,26
5,11,M Gawn,11,4,3,14,1,0.0,34.0,1.0,2.0,5.0,4.0,9.0,2.0,6.0,0.0,12.0,3.0,2.0,0.0,3.0,0.0,0.0,97.0,Melbourne,Western Bulldogs,1,2022,W,26
6,4,J Harmes,6,1,11,17,1,0.0,0.0,5.0,1.0,3.0,2.0,4.0,2.0,1.0,0.0,7.0,10.0,1.0,1.0,3.0,0.0,0.0,77.0,Melbourne,Western Bulldogs,1,2022,W,26
7,29,J Hunt,7,4,4,11,0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,7.0,1.0,0.0,2.0,0.0,0.0,86.0,Melbourne,Western Bulldogs,1,2022,W,26
8,6,L Jackson,3,5,9,12,1,1.0,10.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,8.0,5.0,2.0,1.0,1.0,0.0,0.0,86.0,Melbourne,Western Bulldogs,1,2022,W,26
9,23,J Jordon,10,4,10,20,0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,3.0,0.0,0.0,8.0,12.0,0.0,0.0,0.0,0.0,0.0,70.0,Melbourne,Western Bulldogs,1,2022,W,26


### [1.2 Retrieve web links](#L) <a id='1.2'></a>
To scrape match stats for all the matches we need to retrieve the link for each game's page.

#### [1.2.1 Retrieve web links for 2022 games](#L) <a id='1.2.1'></a>

HTTP Request:

In [19]:
# Get Request
response = requests.get('https://afltables.com/afl/seas/2022.html')

# Status Code check
response.status_code

200

Soup Object:

In [20]:
soup = BeautifulSoup(response.content, 'html.parser')

Retrieve URLs:

In [21]:
url1 = 'https://afltables.com/afl'

# Concatenate the second half of each link to get all 2022 matches
match_stats_urls = [url1 + link.get('href').split('..')[1] for link in soup.findAll('a', href=True, text='Match stats')]

# Drop the last 9 games, which are finals. Brownlow votes are not awarded in finals matches
match_stats_urls = match_stats_urls[:-9]

#### [1.2.2 Retrieve web links for 2015-2022 games](#L) <a id='1.2.2'></a>
The above gets all URLs for all home & away season matches in 2022. Let's get URLs for all home & away season matches over the past 8 years.

In [22]:
url1 = 'https://afltables.com/afl'
match_stats_urls = []

for i in range(2022,2014,-1):
    
    # Get Request
    response = requests.get(f'https://afltables.com/afl/seas/{i}.html')
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Concatenate the second half of each link to get all 2022 matches
    urls = [url1 + link.get('href').split('..')[1] for link in soup.findAll('a', href=True, text='Match stats')]

    # Drop the last 9 games, which are finals. Brownlow votes are not awarded in finals matches
    urls = urls[:-9]
    
    print(f"Number of matches in {i}: {len(urls)}")
    
    match_stats_urls += urls

Number of matches in 2022: 198
Number of matches in 2021: 198
Number of matches in 2020: 153
Number of matches in 2019: 198
Number of matches in 2018: 198
Number of matches in 2017: 198
Number of matches in 2016: 198
Number of matches in 2015: 197


Due to covid, the 2020 season was shortened to 17 rounds while a match was cancelled in 2015 due to the death of Adelaide coach Phil Walsh. 

### [1.3 Wrangle many tables](#L) <a id='1.3'></a>
Now, we create a function to complete the steps in 1.1. Note, there is also a check to make sure that the list index references the correct table, which should have MultiIndex column headers, as mentioned previously. In [this](https://afltables.com/afl/stats/games/2021/071220210402.html) game, there is an additional sundry table before the players stats, which notes a new record for the Western bulldogs, but it does not have a MultiIndex. As a result, the indices of the correct tables are pushed from 2 & 4, to 3 & 5.

In [23]:
def wrangle_stats(page_link):
    '''
    Wrangle the match stats of each player in a game into a single dataframe and clean the data.
    '''
    
    df = pd.read_html(page_link)

    # Set the indices to find team player stats
    idx1 = 2
    
    # If any sundry tables appear on stats page, the correct indices will be pushed down on the page
    while type(df[idx1].columns) != pd.core.indexes.multi.MultiIndex:
        idx1 += 1
    idx2 = idx1 + 2
    
    # Drop top table heading
    df[idx1] = df[idx1].droplevel(0, axis=1)
    df[idx2] = df[idx2].droplevel(0, axis=1)

    # Record club name in each row
    df[idx1]['Club'] = df[0][1][1]
    df[idx2]['Club'] = df[0][1][2]

    # Record opponent club name in each row
    df[idx1]['Opponent'] = df[0][1][2]
    df[idx2]['Opponent'] = df[0][1][1]
    
    # Record year and round in each row
    df[idx1]['Round'] = df[idx2]['Round'] = df[0][1][0].split('Round: ')[1].split(" Venue")[0]
    df[idx1]['Year'] = df[idx2]['Year'] = df[0][1][0].split('-')[2].split(' ')[0]
    
    team1_score = df[0][5][1].split('.')[2]
    team2_score = df[0][5][2].split('.')[2]
    
    # Add result if the player's team won or lost the match
    if team1_score > team2_score:
        df[idx1]['Result'] = 'W'
        df[idx2]['Result'] = 'L'
    elif team2_score > team1_score:
        df[idx1]['Result'] = 'L'
        df[idx2]['Result'] = 'W'
    elif team1_score == team2_score:
        df[idx1]['Result'] = 'D'
        df[idx2]['Result'] = 'D'
    
    # Add game margin
    df[idx1]['Margin'] = int(team1_score) - int(team2_score)
    df[idx2]['Margin'] = int(team2_score) - int(team1_score)
    
    # Drop 'Rushed behind' row and totals rows from the bottom
    df[idx1] = df[idx1][(df[idx1]['#'] != 'Rushed') & (df[idx1]['#'] != 'Totals') & (df[idx1]['#'] != 'Opposition')]
    df[idx2] = df[idx2][(df[idx2]['#'] != 'Rushed') & (df[idx2]['#'] != 'Totals') & (df[idx2]['#'] != 'Opposition')]
    
    # Combine two teams into one dataframe
    df_concat = pd.concat([df[idx1], df[idx2]])
    
    # Remove NaN
    df_concat = df_concat.fillna(0)
    
    # Format player name to first initial, followed by a space, then last name
    df_concat['Player'] = df_concat['Player'].apply(lambda x: f"{x.split(', ')[1][0]} {x.split(', ')[0]}")
    # Format hyphenated last names so that first component is an initial only
    df_concat['Player'] = df_concat['Player'].apply(lambda x: f"{x.split('-')[0][:3]}-{x.split('-')[1]}" if '-' in x else x)
    
    return df_concat

In [24]:
# Get dataframe ready
df = pd.DataFrame(dtype=wrangle_stats(match_stats_urls[0]).info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 22
Data columns (total 31 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   #         46 non-null     object 
 1   Player    46 non-null     object 
 2   KI        46 non-null     object 
 3   MK        46 non-null     object 
 4   HB        46 non-null     object 
 5   DI        46 non-null     object 
 6   GL        46 non-null     object 
 7   BH        46 non-null     float64
 8   HO        46 non-null     float64
 9   TK        46 non-null     float64
 10  RB        46 non-null     float64
 11  IF        46 non-null     float64
 12  CL        46 non-null     float64
 13  CG        46 non-null     float64
 14  FF        46 non-null     float64
 15  FA        46 non-null     float64
 16  BR        46 non-null     float64
 17  CP        46 non-null     float64
 18  UP        46 non-null     float64
 19  CM        46 non-null     float64
 20  MI        46 non-null     float64


In [25]:
# Iterate over each url, run the wrangling function to each match, and combine to the master dataframe
for match in match_stats_urls:
    df = pd.concat([df, wrangle_stats(match)])

In [26]:
df

,#,Player,KI,MK,HB,DI,GL,BH,HO,TK,RB,IF,CL,CG,FF,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P,Club,Opponent,Round,Year,Result,Margin
0,12 ↑,T Bedford,5,3,4,9,0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,4.0,5.0,0.0,1.0,2.0,1.0,0.0,55.0,Melbourne,Western Bulldogs,1,2022,W,26
1,17,J Bowey,8,2,1,9,1,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,6.0,0.0,0.0,2.0,0.0,0.0,73.0,Melbourne,Western Bulldogs,1,2022,W,26
2,10,A Brayshaw,12,6,11,23,0,0.0,0.0,3.0,2.0,2.0,0.0,4.0,1.0,1.0,0.0,4.0,19.0,0.0,0.0,1.0,0.0,0.0,83.0,Melbourne,Western Bulldogs,1,2022,W,26
3,50,B Brown,9,8,4,13,3,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,6.0,7.0,2.0,6.0,2.0,0.0,0.0,86.0,Melbourne,Western Bulldogs,1,2022,W,26
4,31,B Fritsch,8,4,1,9,2,2.0,0.0,1.0,0.0,2.0,0.0,2.0,0.0,1.0,0.0,3.0,5.0,1.0,3.0,0.0,0.0,1.0,81.0,Melbourne,Western Bulldogs,1,2022,W,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,17,J Melksham,10,5,9,19,1,0.0,0.0,2.0,2.0,4.0,3.0,4.0,1.0,2.0,0.0,4.0,14.0,0.0,0.0,0.0,0.0,0.0,86.0,Essendon,Collingwood,23,2015,W,3
18,20,J Merrett,10,5,2,12,1,0.0,0.0,3.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,2.0,11.0,0.0,2.0,0.0,0.0,1.0,84.0,Essendon,Collingwood,23,2015,W,3
19,16,T Pears,6,4,8,14,0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,5.0,10.0,2.0,0.0,5.0,0.0,0.0,88.0,Essendon,Collingwood,23,2015,W,3
20,5,B Stanton,22,8,8,30,3,1.0,0.0,3.0,4.0,5.0,4.0,1.0,0.0,0.0,3.0,7.0,20.0,0.0,1.0,1.0,0.0,0.0,89.0,Essendon,Collingwood,23,2015,W,3


In [27]:
# Reformat Brisbane and GWS club names (for consistency with the next dataset)
df = df.replace({"Club": {"Brisbane Lions": "Brisbane", "Greater Western Sydney":"GWS"},
                 "Opponent": {"Brisbane Lions": "Brisbane", "Greater Western Sydney":"GWS"}})

In [28]:
# Remove arrows next to some player numbers, which represent substituting a player on/off
df['#'] = df['#'].replace({' \u2191':'',' \u2193':''}, regex=True)

## [2. Footywire](#L)<a id='2'></a>
[Footywire](https://www.footywire.com/) has some additional player statistics for games over the past 8 years such as effective disposals and metres gained, which could provide valuable insight into AFL brownlow voting, so we want to retrieve this data and add it to what we already have.

### [2.1 Read and clean tables for one game](#L) <a id='2.1'></a>

In [29]:
# Get Request
response = requests.get('https://www.footywire.com/afl/footy/ft_match_statistics?mid=10544&advv=Y') # First game of 2022

# Status Code check
response.status_code

200

In [30]:
soup = BeautifulSoup(response.content, 'html.parser')

In [31]:
# Get Footywire tables for first game of round 1, 2022
fw = pd.read_html("https://www.footywire.com/afl/footy/ft_match_statistics?mid=10544&advv=Y")

In [32]:
# Fix headings
fw[12].columns = fw[17].columns = fw[12].iloc[0]
fw[12] = fw[12][1:]
fw[17] = fw[17][1:]

In [33]:
fw[12].head(8)

,Player,CP,UP,ED,DE%,CM,GA,MI5,1%,BO,CCL,SCL,SI,MG,TO,ITC,T5,TOG%
1,C Petracca,16,22,29,76.3,0,1,2,0,0,3,6,13,869,4,3,0,87
2,C Oliver,15,20,24,75,0,0,0,2,1,3,3,8,549,8,7,2,81
3,J Viney,12,11,16,69.6,0,0,1,1,0,1,3,5,329,4,7,0,79
4,A Brayshaw,4,19,15,65.2,0,0,0,1,0,0,0,3,264,3,7,0,83
5,E Langdon,11,12,17,77.3,0,1,0,2,1,0,1,6,446,4,7,0,100
6,J Jordon,8,12,16,80,0,0,0,0,0,0,0,2,259,5,8,0,70
7,A N-Bullen,8,10,13,72.2,1,3,0,2,2,0,0,8,320,4,4,1,88
8,J Harmes,7,10,14,82.4,1,0,1,3,0,0,2,4,163,3,2,2,77


If the first team listed on the page activated a substitute during the match, then an additional line appears below that team's statistics table as a footnote. When reading the data using pandas, this will be in the form of another table, which means the index position of the second team will be pushed from 16 to 17.

In [34]:
# Check if arrow icon present in index 13 table (means substitute was activated)
fw[13][0][0][0] == '\u2197' 

True

In [35]:
# Add team names to each row
fw[12]['Club'] = fw[8]['Team'][0]
fw[17]['Club'] = fw[8]['Team'][1]

In [36]:
fw[12].head()

,Player,CP,UP,ED,DE%,CM,GA,MI5,1%,BO,CCL,SCL,SI,MG,TO,ITC,T5,TOG%,Club
1,C Petracca,16,22,29,76.3,0,1,2,0,0,3,6,13,869,4,3,0,87,Melbourne
2,C Oliver,15,20,24,75,0,0,0,2,1,3,3,8,549,8,7,2,81,Melbourne
3,J Viney,12,11,16,69.6,0,0,1,1,0,1,3,5,329,4,7,0,79,Melbourne
4,A Brayshaw,4,19,15,65.2,0,0,0,1,0,0,0,3,264,3,7,0,83,Melbourne
5,E Langdon,11,12,17,77.3,0,1,0,2,1,0,1,6,446,4,7,0,100,Melbourne


In [37]:
# Put both teams together
combined_fw = pd.concat([fw[12], fw[17]])

# Add additional columns (needed for merging dataframes later)
combined_fw['Round'] = soup.find('title').text.split('Round ')[1].split(' ')[0]
combined_fw['Year'] = soup.find('title').text.split(' ')[-1]

In [38]:
# Keep only columns of interest (note we already have some of the statistics from AFL tables)
combined_fw = combined_fw[['Player', 'ED', 'DE%', 'SI', 'MG', 'TO', 'ITC' , 'T5', 'Round', 'Year', 'Club']]

In [39]:
combined_fw

,Player,ED,DE%,SI,MG,TO,ITC,T5,Round,Year,Club
1,C Petracca,29,76.3,13,869,4,3,0,1,2022,Melbourne
2,C Oliver,24,75,8,549,8,7,2,1,2022,Melbourne
3,J Viney,16,69.6,5,329,4,7,0,1,2022,Melbourne
4,A Brayshaw,15,65.2,3,264,3,7,0,1,2022,Melbourne
5,E Langdon,17,77.3,6,446,4,7,0,1,2022,Melbourne
6,J Jordon,16,80,2,259,5,8,0,1,2022,Melbourne
7,A N-Bullen,13,72.2,8,320,4,4,1,1,2022,Melbourne
8,J Harmes,14,82.4,4,163,3,2,2,1,2022,Melbourne
9,M Gawn,8,57.1,8,326,4,4,0,1,2022,Melbourne
10,S May,11,78.6,1,384,3,5,0,1,2022,Melbourne


In [40]:
# Remove arrow symbols next to substitute/d players
combined_fw['Player'] = combined_fw['Player'].replace({' \u2197':'',' \u2199':''}, regex=True)

### [2.2 Retrieve web links](#L) <a id='2.2'></a>
As with AFL tables, we need to retrieve all the URLs for each match page

#### [2.2.1 Retrieve web links for 2022 games](#L) <a id='2.2.1'></a>

HTTP Request:

In [41]:
# Get Request for match summaries page 2022
response = requests.get('https://www.footywire.com/afl/footy/ft_match_list?year=2022')

# Status Code check
response.status_code

200

Soup object:

In [42]:
soup = BeautifulSoup(response.content, 'html.parser')

In [43]:
# We look for all 'tr' tags that have the class 'darkcolor' or 'lightcolor' as this is the pattern for row data in each table
table_rows = soup.findAll('tr', {'class':["darkcolor", 'lightcolor']})

Isolate the relative URLs:

In [44]:
# Narrow the search down further
table_rows[0].findAll('td', {'class': 'data'})

[<td class="data" height="24"> Wed 16 Mar 7:10pm</td>,
 <td class="data">
 <a href="th-melbourne-demons">Melbourne</a>
 v 
 <a href="th-western-bulldogs">Western Bulldogs</a>
 </td>,
 <td class="data">MCG</td>,
 <td align="center" class="data">58002</td>,
 <td align="center" class="data"><a href="ft_match_statistics?mid=10544">97-71</a></td>,
 <td class="data">
 <a href="ft_player_profile?pid=3800">J. Macrae</a> 39<br/>
 </td>,
 <td class="data">
 <a href="ft_player_profile?pid=6491">A. Naughton</a> 4<br/>
 </td>]

The link to the page we want in each row is in the 5th column (index 4), so we search for td tag, class data, index 4 and specify that we want the relative url.

In [45]:
table_rows[0].findAll('td', {'class': 'data'})[4].find('a').get('href')

'ft_match_statistics?mid=10544'

Then we store all the URLs in a list, concatenating the first part of the URL with relative URL for each page.

In [46]:
fw_urls = []
url_p1 = "https://www.footywire.com/afl/footy/"

# Find url to advanced stats page for each home and away match in 2022 season, ignore any invalid rows
for row in table_rows:
    try:
        fw_urls.append(f"{url_p1}{row.findAll('td', {'class': 'data'})[4].find('a').get('href')}&advv=Y")
    except:
        pass

In [47]:
fw_urls = fw_urls[:-9] # Cut out finals
fw_urls[:5]

['https://www.footywire.com/afl/footy/ft_match_statistics?mid=10544&advv=Y',
 'https://www.footywire.com/afl/footy/ft_match_statistics?mid=10545&advv=Y',
 'https://www.footywire.com/afl/footy/ft_match_statistics?mid=10546&advv=Y',
 'https://www.footywire.com/afl/footy/ft_match_statistics?mid=10547&advv=Y',
 'https://www.footywire.com/afl/footy/ft_match_statistics?mid=10548&advv=Y']

#### [2.2.2 Retrieve web links for 2015-2022 games](#L) <a id='2.2.2'></a>

In [48]:
table_rows = []

for i in range(2022,2014,-1):
    
    # Get Request
    response = requests.get(f'https://www.footywire.com/afl/footy/ft_match_list?year={i}')
    # Create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    # Get table rows data
    table_rows += soup.findAll('tr', {'class':["darkcolor", 'lightcolor']})

In [49]:
fw_urls = []
url_p1 = "https://www.footywire.com/afl/footy/"

# Find url to advanced stats page for each home and away match in 2022 season
for row in table_rows:
    try:
        fw_urls.append(f"{url_p1}{row.findAll('td', {'class': 'data'})[4].find('a').get('href')}&advv=Y")
    except:
        pass

In [50]:
len(fw_urls)

1610

### [2.3 Wrangle many tables](#L) <a id='2.3'> </a>

In [51]:
def wrangle_fw_stats(url):
    """
    Wrangle the footywire statistics
    """

    # Get Footywire tables
    fw = pd.read_html(url)
    
    # If substitute activated for team 1, the website shows an additional line below table 1, which pushses
    # the index for team 2 down to 17, otherwise it will be 16. '\u2197' is the unicode for the arrow symbol
    # that can be used to determine if team 1 activated a substitute in that match.
    if fw[13][0][0][0] == '\u2197':
        t2_idx = 17
    else:
        t2_idx = 16
    
    # Fix headings
    fw[12].columns = fw[t2_idx].columns = fw[12].iloc[0]
    fw[12] = fw[12][1:]
    fw[t2_idx] = fw[t2_idx][1:]
    
    # Add team names to each row
    fw[12]['Club'] = fw[8]['Team'][0]
    fw[t2_idx]['Club'] = fw[8]['Team'][1]
    
    # Put both teams together
    combined_fw = pd.concat([fw[12], fw[t2_idx]])
    
    # Add additional columns (for merging dataframes)
    combined_fw['Round'] = fw[7][0][1].split(',')[0].split(' ')[1]
    combined_fw['Year'] = fw[7][0][2].split(',')[1].split(' ')[3]
    
    # Keep only columns of interest (note we already have some of the statistics from AFL tables)
    combined_fw = combined_fw[['Player', 'ED', 'DE%', 'SI', 'MG', 'TO', 'ITC' , 'T5', 'Round', 'Year', 'Club']]
    
    # Remove arrow symbols next to substitute/d players
    combined_fw['Player'] = combined_fw['Player'].replace({' \u2197':'',' \u2199':''}, regex=True)
    
    return combined_fw

In [52]:
# Get dataframe ready
fw = pd.DataFrame(dtype=wrangle_fw_stats(fw_urls[0]).info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 1 to 23
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  46 non-null     object
 1   ED      46 non-null     object
 2   DE%     46 non-null     object
 3   SI      46 non-null     object
 4   MG      46 non-null     object
 5   TO      46 non-null     object
 6   ITC     46 non-null     object
 7   T5      46 non-null     object
 8   Round   46 non-null     object
 9   Year    46 non-null     object
 10  Club    46 non-null     object
dtypes: object(11)
memory usage: 4.3+ KB


In [53]:
%%time

# Run wrangling function over each table and concatenate together
for url in fw_urls:
    fw = pd.concat([fw, wrangle_fw_stats(url)])

Wall time: 1h 12min 46s


In [54]:
fw

,Player,ED,DE%,SI,MG,TO,ITC,T5,Round,Year,Club
1,C Petracca,29,76.3,13,869,4,3,0,1,2022,Melbourne
2,C Oliver,24,75,8,549,8,7,2,1,2022,Melbourne
3,J Viney,16,69.6,5,329,4,7,0,1,2022,Melbourne
4,A Brayshaw,15,65.2,3,264,3,7,0,1,2022,Melbourne
5,E Langdon,17,77.3,6,446,4,7,0,1,2022,Melbourne
...,...,...,...,...,...,...,...,...,...,...,...
18,X Ellis,8,88.9,2,126,0,3,0,Final,2015,West Coast
19,J Kennedy,6,66.7,5,123,1,0,0,Final,2015,West Coast
20,E Yeo,5,100,2,102,2,1,0,Final,2015,West Coast
21,N Naitanui,1,25,4,132,1,0,0,Final,2015,West Coast


In [55]:
# Remove string values for player that did not get on the ground, to 0
fw = fw.replace("Unused Substitute", 0)

## [3. Merge "AFL Tables" and "Footywire" dataframes](#L) <a id = '3'></a>

In [56]:
# Merge two dataframes on several columns
joined = pd.merge(df, fw, how='outer', indicator='True', on=['Player', 'Club', 'Round', 'Year'])

In [57]:
joined

,#,Player,KI,MK,HB,DI,GL,BH,HO,TK,RB,IF,CL,CG,FF,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P,Club,Opponent,Round,Year,Result,Margin,ED,DE%,SI,MG,TO,ITC,T5,True
0,12,T Bedford,5,3,4,9,0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,4.0,5.0,0.0,1.0,2.0,1.0,0.0,55.0,Melbourne,Western Bulldogs,1,2022,W,26.0,7,77.8,1,124,1,4,1,both
1,17,J Bowey,8,2,1,9,1,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,6.0,0.0,0.0,2.0,0.0,0.0,73.0,Melbourne,Western Bulldogs,1,2022,W,26.0,8,88.9,1,197,2,4,0,both
2,10,A Brayshaw,12,6,11,23,0,0.0,0.0,3.0,2.0,2.0,0.0,4.0,1.0,1.0,0.0,4.0,19.0,0.0,0.0,1.0,0.0,0.0,83.0,Melbourne,Western Bulldogs,1,2022,W,26.0,15,65.2,3,264,3,7,0,both
3,50,B Brown,9,8,4,13,3,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,6.0,7.0,2.0,6.0,2.0,0.0,0.0,86.0,Melbourne,Western Bulldogs,1,2022,W,26.0,8,61.5,11,245,1,0,0,both
4,31,B Fritsch,8,4,1,9,2,2.0,0.0,1.0,0.0,2.0,0.0,2.0,0.0,1.0,0.0,3.0,5.0,1.0,3.0,0.0,0.0,1.0,81.0,Melbourne,Western Bulldogs,1,2022,W,26.0,5,55.6,6,251,2,0,0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73010,NaN,M Taberner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fremantle,NaN,Final,2015,NaN,NaN,4,50,2,106,4,0,1,right_only
73011,NaN,M Priddis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Coast,NaN,Final,2015,NaN,NaN,18,69.2,9,318,2,0,0,right_only
73012,NaN,M Priddis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Coast,NaN,Final,2015,NaN,NaN,18,72,6,256,5,2,3,right_only
73013,NaN,C Masten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Coast,NaN,Final,2015,NaN,NaN,9,90,4,106,2,0,0,right_only


There were a few rows that failed to merge, due to differences in formatting or because of finals matches included in the second dataset:

In [58]:
joined[joined['True']=='right_only']['Round'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       'Final', 'Finals'], dtype=object)

In [59]:
# Drop finals rounds
joined = joined[(joined['Round']!='Final')&(joined['Round']!='Finals')]

In [60]:
joined[joined['True']=='left_only']['Player'].unique()

array(['L OBrien', 'J de Goey', 'M de Boer', 'X OHalloran', 'C ORiordan',
       'C Macdonald', 'J OMeara', 'R OBrien', 'X ONeill', 'K McIntosh',
       'N ODriscoll', 'M OConnor', 'T OBrien', 'M DAmbrosio',
       'H Paul Oea', 'D Mackay', 'R OConnor', 'J Macmillan',
       'J Kennedy Harris', 'M LeCras', 'C OShea', 'J ORourke',
       'E Mackenzie', 'J Deluca', 'N van Berlo', 'L McBean', 'Z OBrien',
       'M McDonough', 'N OBrien'], dtype=object)

In [61]:
joined[joined['True']=='right_only']['Player'].unique()

array(["L O'Brien", 'J De Goey', 'M De Boer', "X O'Halloran",
       "C O'Riordan", 'C MacDonald', "J O'Meara", "R O'Brien",
       "X O'Neill", "N O'Driscoll", 'K Mcintosh', "M O'Connor",
       "T O'Brien", "M D'Ambrosio", 'H Oea', 'D MacKay', "R O'Connor",
       'J MacMillan', 'J K-Harris', 'M Lecras', "C O'Shea", "J O'Rourke",
       'E MacKenzie', 'J D-Cardillo', 'N Van Berlo', 'L Mcbean',
       "Z O'Brien", 'M Mcdonough', "N O'Brien"], dtype=object)

There are still some formatting inconsistencies in the player names of the two datasets, which are manually fixed below. Most could also be fixed by making all player names entirely lower/uppercase and removing apostrophes.

In [77]:
df = df.replace({"Player": {
    "L OBrien" : "L O'Brien",
    "J de Goey" : "J De Goey",
    "M de Boer" : "M De Boer",
    "X OHalloran" : "X O'Halloran",
    "C ORiordan" : "C O'Riordan",
    "C Macdonald" : "C MacDonald",
    "J OMeara" : "J O'Meara",
    "R OBrien" : "R O'Brien",
    "X ONeill" : "X O'Neill",
    "N ODriscoll" : "N O'Driscoll",
    "M OConnor" : "M O'Connor",
    "T OBrien" : "T O'Brien",
    "M DAmbrosio" : "M D'Ambrosio",
    "H Paul Oea" : "H Oea",
    "R OConnor" : "R O'Connor",
    "C OShea" : "C O'Shea",
    "J ORourke" : "J O'Rourke",
    "Z OBrien" : "Z O'Brien",
    "N OBrien" : "N O'Brien",
    }})

fw = fw.replace({"Player": {
    "K Mcintosh": "K McIntosh", 
    "D MacKay": "D Mackay",
    "J MacMillan" : "J Macmillan",
    "J K-Harris" : "J Kennedy Harris",
    "M Lecras" : "M LeCras",
    "E MacKenzie" : "E Mackenzie",
    "N Van Berlo" : "N van Berlo",
    "L Mcbean" : "L McBean",
    "M Mcdonough" : "M McDonough",
    "J D-Cardillo" : "J Deluca",
    }})

Now we can merge the dataframes again.

In [78]:
# Merge two datasets, exluding finals stats
joined = pd.merge(df, fw, how='outer', indicator='True', on=['Player', 'Club', 'Round', 'Year'])
joined = joined[(joined['Round']!='Final')&(joined['Round']!='Finals')]

In [79]:
# Check no mismatches between left and right dataframes
joined[joined['True']=='left_only']['Player'].unique()

array([], dtype=object)

In [80]:
# Check no mismatches between left and right dataframes
joined[joined['True']=='right_only']['Player'].unique()

array([], dtype=object)

In [81]:
# Drop redundant column
joined = joined.drop('True', axis=1)

In [82]:
joined

,#,Player,KI,MK,HB,DI,GL,BH,HO,TK,RB,IF,CL,CG,FF,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P,Club,Opponent,Round,Year,Result,Margin,ED,DE%,SI,MG,TO,ITC,T5
0,12,T Bedford,5,3,4,9,0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,4.0,5.0,0.0,1.0,2.0,1.0,0.0,55.0,Melbourne,Western Bulldogs,1,2022,W,26.0,7,77.8,1,124,1,4,1
1,17,J Bowey,8,2,1,9,1,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,6.0,0.0,0.0,2.0,0.0,0.0,73.0,Melbourne,Western Bulldogs,1,2022,W,26.0,8,88.9,1,197,2,4,0
2,10,A Brayshaw,12,6,11,23,0,0.0,0.0,3.0,2.0,2.0,0.0,4.0,1.0,1.0,0.0,4.0,19.0,0.0,0.0,1.0,0.0,0.0,83.0,Melbourne,Western Bulldogs,1,2022,W,26.0,15,65.2,3,264,3,7,0
3,50,B Brown,9,8,4,13,3,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,6.0,7.0,2.0,6.0,2.0,0.0,0.0,86.0,Melbourne,Western Bulldogs,1,2022,W,26.0,8,61.5,11,245,1,0,0
4,31,B Fritsch,8,4,1,9,2,2.0,0.0,1.0,0.0,2.0,0.0,2.0,0.0,1.0,0.0,3.0,5.0,1.0,3.0,0.0,0.0,1.0,81.0,Melbourne,Western Bulldogs,1,2022,W,26.0,5,55.6,6,251,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68459,17,J Melksham,10,5,9,19,1,0.0,0.0,2.0,2.0,4.0,3.0,4.0,1.0,2.0,0.0,4.0,14.0,0.0,0.0,0.0,0.0,0.0,86.0,Essendon,Collingwood,23,2015,W,3.0,14,73.7,2,430,6,4,0
68460,20,J Merrett,10,5,2,12,1,0.0,0.0,3.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,2.0,11.0,0.0,2.0,0.0,0.0,1.0,84.0,Essendon,Collingwood,23,2015,W,3.0,10,83.3,5,206,3,3,0
68461,16,T Pears,6,4,8,14,0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,5.0,10.0,2.0,0.0,5.0,0.0,0.0,88.0,Essendon,Collingwood,23,2015,W,3.0,9,64.3,3,63,3,6,0
68462,5,B Stanton,22,8,8,30,3,1.0,0.0,3.0,4.0,5.0,4.0,1.0,0.0,0.0,3.0,7.0,20.0,0.0,1.0,1.0,0.0,0.0,89.0,Essendon,Collingwood,23,2015,W,3.0,22,73.3,9,645,5,4,1


In [83]:
# Alter column order
cols = joined.columns.tolist()

cols = cols[:2] + cols[25:27] + cols[28:26:-1] + cols[29:31] + [cols[2]] + cols[4:6] + cols[31:33] + [cols[3]] + cols[6:8] + \
[cols[23]] + [cols[33]] + cols[8:16] + cols[17:23] + [cols[24]] + cols[-4:] + [cols[16]]

joined = joined[cols]

In [84]:
joined

,#,Player,Club,Opponent,Year,Round,Result,Margin,KI,HB,DI,ED,DE%,MK,GL,BH,GA,SI,HO,TK,RB,IF,CL,CG,FF,FA,CP,UP,CM,MI,1%,BO,%P,MG,TO,ITC,T5,BR
0,12,T Bedford,Melbourne,Western Bulldogs,2022,1,W,26.0,5,4,9,7,77.8,3,0,1.0,0.0,1,0.0,2.0,1.0,1.0,0.0,1.0,2.0,0.0,4.0,5.0,0.0,1.0,2.0,1.0,55.0,124,1,4,1,0.0
1,17,J Bowey,Melbourne,Western Bulldogs,2022,1,W,26.0,8,1,9,8,88.9,2,1,0.0,0.0,1,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,3.0,6.0,0.0,0.0,2.0,0.0,73.0,197,2,4,0,0.0
2,10,A Brayshaw,Melbourne,Western Bulldogs,2022,1,W,26.0,12,11,23,15,65.2,6,0,0.0,0.0,3,0.0,3.0,2.0,2.0,0.0,4.0,1.0,1.0,4.0,19.0,0.0,0.0,1.0,0.0,83.0,264,3,7,0,0.0
3,50,B Brown,Melbourne,Western Bulldogs,2022,1,W,26.0,9,4,13,8,61.5,8,3,3.0,0.0,11,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,6.0,7.0,2.0,6.0,2.0,0.0,86.0,245,1,0,0,0.0
4,31,B Fritsch,Melbourne,Western Bulldogs,2022,1,W,26.0,8,1,9,5,55.6,4,2,2.0,1.0,6,0.0,1.0,0.0,2.0,0.0,2.0,0.0,1.0,3.0,5.0,1.0,3.0,0.0,0.0,81.0,251,2,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68459,17,J Melksham,Essendon,Collingwood,2015,23,W,3.0,10,9,19,14,73.7,5,1,0.0,0.0,2,0.0,2.0,2.0,4.0,3.0,4.0,1.0,2.0,4.0,14.0,0.0,0.0,0.0,0.0,86.0,430,6,4,0,0.0
68460,20,J Merrett,Essendon,Collingwood,2015,23,W,3.0,10,2,12,10,83.3,5,1,0.0,1.0,5,0.0,3.0,0.0,2.0,1.0,1.0,1.0,0.0,2.0,11.0,0.0,2.0,0.0,0.0,84.0,206,3,3,0,0.0
68461,16,T Pears,Essendon,Collingwood,2015,23,W,3.0,6,8,14,9,64.3,4,0,0.0,0.0,3,0.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,5.0,10.0,2.0,0.0,5.0,0.0,88.0,63,3,6,0,0.0
68462,5,B Stanton,Essendon,Collingwood,2015,23,W,3.0,22,8,30,22,73.3,8,3,1.0,0.0,9,0.0,3.0,4.0,5.0,4.0,1.0,0.0,0.0,7.0,20.0,0.0,1.0,1.0,0.0,89.0,645,5,4,1,3.0


In [85]:
# Export to csv
joined.to_csv("afl_tables.csv", index=False)